In [97]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math

from utility_code.utility import utils

In [130]:
#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")
dummydata = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')

In [131]:
dummydata = dummydata[dummydata['Location'].isin(train['Province_State'])]


In [132]:
dummydata['Population'] = dummydata['Number of COVID-19 Cases'].divide(dummydata['COVID-19 Cases per 1,000,000 Population']) * 1e6

In [133]:
dummydata

,Location,Number of COVID-19 Cases,"COVID-19 Cases per 1,000,000 Population",Deaths from COVID-19,"COVID-19 Deaths per 1,000,000 Population",COVID-19 Fatality Rate,Footnotes,Population
1,Alabama,236865.0,48308.0,3472.0,708.0,0.014658,NaN,4.903225e+06
2,Alaska,28273.0,38648.0,103.0,141.0,0.003643,NaN,7.315514e+05
3,Arizona,302324.0,41535.0,6464.0,888.0,0.021381,NaN,7.278777e+06
4,Arkansas,146190.0,48443.0,2387.0,791.0,0.016328,NaN,3.017773e+06
5,California,1135761.0,28745.0,18784.0,475.0,0.016539,NaN,3.951160e+07
6,Colorado,202289.0,35127.0,2810.0,488.0,0.013891,NaN,5.758790e+06
7,Connecticut,106740.0,29939.0,4871.0,1366.0,0.045634,NaN,3.565249e+06
8,Delaware,32664.0,33544.0,757.0,777.0,0.023175,NaN,9.737658e+05
10,Florida,944745.0,43987.0,18085.0,842.0,0.019143,NaN,2.147782e+07
11,Georgia,451056.0,42483.0,9215.0,868.0,0.020430,NaN,1.061733e+07


In [159]:
statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [160]:
def deriv(y, t, N, beta, gamma, delta):
    S, I, R, D = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - (gamma+delta) * I 
    dRdt = gamma * I
    dDdt = delta * I
    return dSdt, dIdt, dRdt, dDdt

In [191]:
projections = {}
days_to_forecast = 26
for s in states:
#     print('state', s)
    ala = statesdata[s]

    foundfirst = False
    first = -1
    second = -1
    for val in ala['Recovered'].unique():
        if val == 'na' or val != val or val == 0:
            continue
        if not foundfirst: 
            first = val
            foundfirst = True
            continue
        second = val
    if first != -1 and second != -1:
        #print(first, second)
        dur = int(ala.loc[ala['Recovered'] == second,'Date'].iloc[0].split('-')[1]) - int(ala.loc[ala['Recovered'] == first, 'Date'].iloc[0].split('-')[1])
        gamma = (second-first)/dur
        act = ala.loc[ala['Recovered'] == second,'Active'].iloc[0]
        gamma /= act

    delta = float(dummydata.loc[dummydata['Location'] == s]['COVID-19 Fatality Rate'])
    b = ala['Active'].iloc[1] - ala['Active'].iloc[0]
    b = b/ala['Active'].iloc[0]
    pop = int(dummydata.loc[dummydata['Location'] == s,'Population'])
    beta = b * pop/(pop - (ala['Confirmed'].iloc[0]))
        
    I0, R0, D0 = ala.iloc[len(ala)-1,6], ala.iloc[len(ala)-1, 5], ala.iloc[len(ala)-1,4]
    if I0 != I0:
        I0 = 0
    if R0 != R0:
        R0 = 0
    if D0 != 0:
        D0 = 0 
    S0 = pop - I0 - R0 - D0
    t = np.linspace(1, days_to_forecast, days_to_forecast)
    y0 = S0, I0, R0, D0
    
    if s == 'California':
        print('beta, gamma, delta', beta, gamma, delta)
        print('i, r, d, s', I0, R0, D0, S0)
        print('t', t)

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(pop, beta, gamma, delta))
    S, I, R, D = ret.T
    
    y = [sum(x) for x in zip(I.tolist(), R.tolist())]
    y = [sum(x) for x in zip(y, D.tolist())]
    d = D.tolist()
    
    y = [int(i) for i in y]
    d = [int(j) for j in d]
    toreturn = {'confirmed': y,'deaths': d}
    
    projections[s] = toreturn
    
    
    
    
    

beta, gamma, delta 0.047962673970532194 0.5256802819342539 0.016538690800000002
i, r, d, s 699453.0 0 0 38812149.0
t [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26.]


/Users/Aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Aditya/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


ValueError: cannot convert float NaN to integer

In [53]:
float(dummydata.loc[dummydata['Location'] == 'Alabama']['COVID-19 Fatality Rate'])

0.0146581386

In [180]:
ala = statesdata['California']

foundfirst = False

for val in ala['Recovered'].unique():
    if val == 'na' or val != val or val == 0:
        continue
    if not foundfirst: 
        first = val
        foundfirst = True
        continue
    second = val

# print(first, second)
dur = int(ala.loc[ala['Recovered'] == second,'Date'].iloc[0].split('-')[1]) - int(ala.loc[ala['Recovered'] == first, 'Date'].iloc[0].split('-')[1])
gamma = (second-first)/dur
act = ala.loc[ala['Recovered'] == second,'Active'].iloc[0]
gamma /= act
print(gamma)

IndexError: single positional indexer is out-of-bounds

In [164]:
ala['Recovered'].unique()

array([nan,  0.])

In [153]:
ala.iloc[(len(ala)-1),6]

75848.0

In [144]:
b = ala['Active'].iloc[-1] - ala['Active'].iloc[-2]
b = b/ala['Active'].iloc[-2]
pop = int(dummydata.loc[dummydata['Location'] == 'Alabama','Population'])
b = b * pop/(pop - (ala['Confirmed'].iloc[-2]))
print(b)

0.010980708673924498


In [194]:
projections['California']['confirmed']

[699453,
 725435,
 741262,
 750905,
 756781,
 760362,
 762544,
 763874,
 764684,
 765178,
 765479,
 765663,
 765775,
 765843,
 765884,
 765910,
 765925,
 765934,
 765940,
 765944,
 765946,
 765947,
 765948,
 765948,
 765949,
 765949]

In [182]:
ala = statesdata['California']
ala

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
4,4,California,04-12-2020,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,2.812020,485.423869,22.961176
54,54,California,04-13-2020,23931,714,NaN,23217.0,61.035048,190882.0,3015.0,2.987756,486.836823,12.598721
104,104,California,04-14-2020,25356,767,NaN,24589.0,64.669453,202208.0,5163.0,3.028869,515.723328,20.362044
154,154,California,04-15-2020,26686,860,NaN,25826.0,68.061564,216486.0,5163.0,3.226411,552.138790,19.347223
204,204,California,04-16-2020,27677,956,NaN,26721.0,70.589069,246400.0,5031.0,3.457745,628.433237,18.177548
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854,6854,California,08-27-2020,691821,12677,NaN,679144.0,1750.903765,10918415.0,NaN,1.832410,27633.006121,NaN
6904,6904,California,08-28-2020,697385,12805,NaN,684580.0,1764.985483,11010637.0,NaN,1.836145,27866.407314,NaN
6954,6954,California,08-29-2020,702038,12894,NaN,689144.0,1776.761586,11109630.0,NaN,1.836653,28116.944977,NaN
7004,7004,California,08-30-2020,705951,12937,NaN,693014.0,1786.664851,11231829.0,NaN,1.832563,28426.213833,NaN
